In [31]:
from bertopic import BERTopic
import pandas as pd
from pathlib import Path

repo_path = Path('/home/krajda/anticipatio/')

topic_model = BERTopic.load(repo_path / 'models/pca10_kmeans200_cv.pkl')
tweets = pd.read_pickle(repo_path / 'data/final.pkl')
docs = tweets['txt'].tolist()

In [3]:
import itertools
import pandas as pd
from pathlib import Path

def open_fn(f):
    try:
        return pd.read_csv(f,engine='python')
    except:
        return pd.DataFrame()
    
files1 = Path(repo_path / 'data/futurists_kol/data').rglob('*csv')
files2 = Path(repo_path / 'data/futurists_rossdawson/data').rglob('*csv')
files = itertools.chain(files1,files2)
tweets = map(open_fn, files)
tweets = pd.concat(tweets)

tweets.columns = ['index','user','timestamp','url','txt']
tweets.reset_index(drop=True,inplace=True)
tweets['txt'] = tweets['txt'].astype(str)
tweets['user']=tweets['user'].str.replace('@','').str.strip().str.lower()
tweets['timestamp'] = pd.to_datetime(tweets['timestamp'])
tweets.drop(columns='index', inplace=True)
tweets.drop_duplicates(inplace=True,subset=['timestamp','txt'])
tweets.reset_index(inplace=True,drop=True)
print('Unique users:{}\nUnique texts:{}'.format(tweets['user'].nunique(),tweets['txt'].nunique()))
maciek_docs = tweets['txt']
print('Unique posts (timestamp+text): {}'.format(len(maciek_docs)))

Unique users:400
Unique texts:1200003
Unique posts (timestamp+text): 1458018


In [24]:
print('Unique posts (timestamp+text): {}'.format(len(maciek_docs)))

Unique posts (timestamp+text): 1458018


In [23]:
maciek_labels = pd.read_csv(repo_path / 'data/topic_labels.csv', header=None)

maciek_labels = maciek_labels[1][1:].tolist()
maciek_labels



[50,
 50,
 27,
 46,
 6,
 69,
 40,
 89,
 27,
 69,
 24,
 49,
 31,
 62,
 51,
 48,
 65,
 90,
 20,
 90,
 32,
 6,
 76,
 9,
 0,
 59,
 80,
 98,
 26,
 80,
 55,
 60,
 57,
 44,
 0,
 7,
 74,
 93,
 25,
 13,
 7,
 43,
 33,
 9,
 52,
 41,
 51,
 69,
 89,
 98,
 59,
 83,
 12,
 56,
 18,
 87,
 5,
 19,
 47,
 11,
 87,
 53,
 40,
 43,
 45,
 87,
 52,
 97,
 25,
 18,
 27,
 65,
 69,
 66,
 13,
 0,
 10,
 69,
 13,
 65,
 78,
 85,
 59,
 53,
 70,
 68,
 91,
 37,
 64,
 65,
 18,
 79,
 56,
 2,
 82,
 40,
 88,
 91,
 34,
 20,
 28,
 36,
 97,
 73,
 52,
 51,
 0,
 74,
 65,
 55,
 7,
 6,
 16,
 50,
 0,
 48,
 83,
 93,
 92,
 47,
 12,
 16,
 23,
 16,
 28,
 72,
 97,
 65,
 35,
 94,
 75,
 65,
 93,
 46,
 87,
 45,
 53,
 15,
 59,
 37,
 0,
 94,
 24,
 55,
 45,
 94,
 11,
 28,
 40,
 5,
 76,
 15,
 63,
 56,
 43,
 97,
 8,
 19,
 69,
 94,
 11,
 35,
 54,
 5,
 23,
 95,
 88,
 24,
 40,
 49,
 31,
 64,
 21,
 70,
 8,
 0,
 52,
 85,
 67,
 72,
 3,
 95,
 26,
 88,
 84,
 88,
 69,
 7,
 84,
 67,
 56,
 10,
 25,
 89,
 92,
 10,
 83,
 8,
 42,
 18,
 59,
 17,
 83,
 13,
 24,

In [6]:
maciek_docs = maciek_docs.tolist()

In [9]:
import json

with open(repo_path / 'data/labels_mapping.json', 'r') as f:
    mapping = json.load(f)

In [27]:
c = 0

for m in mapping:
    
    if len(m) > 1:
        c += 1
        

In [28]:
c

33396

In [29]:
mapped_labels = [maciek_labels[m[0]] for m in mapping]

In [32]:
tweets['Topic'] = mapped_labels

In [33]:
tweets.rename(columns={'emotion': 'b24_emotion'}, inplace=True)

emotions = pd.read_pickle('/home/krajda/anticipatio/data/emotions.pkl')

In [34]:
tweets = tweets.reset_index()
tweets['id'] = tweets.index
emotions['id'] = emotions.index

tweets = tweets.join(emotions.set_index('id'), on='id')

In [35]:
agg = tweets.groupby('Topic').mean().drop(columns=['id', 'index'])

In [70]:
agg

,sadness,disgust,anger,pessimism,fear,anticipation,surprise,joy,optimism,love,trust
Topic,,,,,,,,,,,
0,0.095987,0.141452,0.129143,0.053873,0.075708,0.429609,0.086598,0.508406,0.473497,0.088471,0.122304
1,0.097344,0.142946,0.130451,0.054166,0.073885,0.429430,0.087059,0.508743,0.473417,0.090189,0.123260
2,0.098241,0.143166,0.130882,0.054149,0.075918,0.430448,0.086806,0.502735,0.472931,0.087960,0.121895
3,0.096948,0.146060,0.133718,0.053755,0.074449,0.428688,0.086623,0.505320,0.467507,0.086453,0.121800
4,0.098111,0.143248,0.130306,0.055020,0.076126,0.430150,0.087068,0.504335,0.473110,0.087553,0.122776
...,...,...,...,...,...,...,...,...,...,...,...
95,0.097636,0.144270,0.131918,0.054181,0.073345,0.428977,0.086424,0.503757,0.472814,0.087058,0.122578
96,0.094765,0.144439,0.132261,0.053010,0.074281,0.430692,0.086527,0.508169,0.471663,0.090012,0.122310
97,0.094923,0.142620,0.130936,0.054089,0.076335,0.433905,0.087624,0.511982,0.476413,0.090519,0.124604


In [41]:
agg.to_csv(repo_path / 'data/to_paper/emotions_per_topic.csv')

In [91]:
anticipated_representatives = []

examples = 20

for n in range(100):
    x = tweets[tweets['Topic'] == n].nlargest(examples, ['anticipation'])[['anticipation', 'txt']]
    
    a = x['anticipation'].tolist()
    t = x['txt'].tolist()

    for i in range(examples):
        anticipated_representatives.append({'topic': n, 'anticipation': a[i], 'text': t[i]})
    
    
pd.DataFrame(anticipated_representatives).to_csv(repo_path / 'data/to_paper/anticipated_representatives.csv')

In [67]:
anticipated_representatives

,topic,anticipation,text
0,0,0.924846,Let me know if you're ever interested in out-o...
1,0,0.920147,Hey clive. Can you email me about a project idea?
2,0,0.918372,It will be interesting to see what the score i...
3,0,0.918061,Looking forward to seeing how Dune II reinvent...
4,0,0.914009,Now how about for a guest interview for a show
5,0,0.911984,Who would beat you more quickly in a game of c...
6,0,0.911572,Plenty of supply in the Bay Area. The question...
7,0,0.910785,AR will matter next Christmas. Wanna talk abou...
8,0,0.909462,I'm interested in joining as a trainer. Can an...
9,0,0.908846,Lucid piece Alex. Can’t wait for the rest of ...


In [81]:
tweets['x'] = 1
topic_user = tweets[['Topic', 'user', 'x']].groupby(['Topic', 'user'], as_index=False)['x'].count().sort_values(by=['Topic','x'], ascending=False)

In [95]:
key_opinion_leaders = []

examples = 10

for n in range(100):
    x = topic_user[topic_user['Topic'] == n].nlargest(examples, ['x'])[['x', 'user']]
    
    a = x['x'].tolist()
    t = x['user'].tolist()

    for i in range(examples):
        key_opinion_leaders.append({'topic': n, 'tweets_count': a[i], 'user': t[i]})
        
    

pd.DataFrame(key_opinion_leaders).to_csv(repo_path / 'data/to_paper/key_opinion_leaders.csv')

In [87]:
topic_user[topic_user['user'] == '@EvanKirstel']['x'].sum()

77428

In [93]:
tweets['user'].value_counts(sort=True, ascending=False)[:10]

@EvanKirstel        77428
@digital_trans4m    37445
@psb_dc             36311
@KirkDBorne         33840
@jamesvgingerich    31682
@Ronald_vanLoon     30880
scobleizer          30244
@TechCrunch         29518
@ipfconline1        29341
@rwang0             26916
Name: user, dtype: int64

In [104]:
users_sum = topic_user.groupby('user')['x'].sum().reset_index()
users_sum.sort_values(by='x', ascending=False, inplace=True)
users_sum

,user,x
23,@EvanKirstel,77428
128,@digital_trans4m,37445
209,@psb_dc,36311
46,@KirkDBorne,33840
153,@jamesvgingerich,31682
...,...,...
40,@JesusSanSegundo,1
147,@ingliguori,1
361,localrat,1
6,@BigDataBRA,1


In [105]:
topic_user['share'] = topic_user.apply(lambda x: x['x'] / users_sum[users_sum['user'] == x['user']]['x'].values[0], axis=1)

In [131]:
topic_user['user_sum'] = topic_user.apply(lambda x: users_sum[users_sum['user'] == x['user']]['x'].values[0], axis=1)

In [137]:
topic_user2 = topic_user[topic_user['user_sum'] > 10]

topic_user2.groupby('user')['share'].max().reset_index().sort_values(by='share', ascending=False)

,user,share
98,@bethnoveck,0.217391
125,@ericamarois,0.187500
317,futurprospicere,0.157895
369,rodfalcon,0.142857
121,@didiebon,0.133333
...,...,...
271,StephaniePride,0.019026
236,@wef,0.018743
289,bronwynwilliams,0.018591
274,Tiffani_Bova,0.018480


In [134]:
topic_user2[topic_user['user'] == '@bethnoveck']

,Topic,user,x,share,user_sum
33034,99,@bethnoveck,1,0.043478,23
31709,95,@bethnoveck,1,0.043478,23
31378,94,@bethnoveck,1,0.043478,23
27694,83,@bethnoveck,1,0.043478,23
27024,81,@bethnoveck,1,0.043478,23
24754,74,@bethnoveck,1,0.043478,23
23137,69,@bethnoveck,1,0.043478,23
22815,68,@bethnoveck,1,0.043478,23
21472,64,@bethnoveck,2,0.086957,23
16486,49,@bethnoveck,1,0.043478,23
